# Project: Dynamics of Bargaining Pwer
# Standard Static Model (JuMP)
#### Genaro, Itza, & Sonia
#### May 2021 

This code was made specifically to solve the principal-agent problem in a static single objective model. 

This aplication uses the JuMP library and the Ipopt solver to solve the optimization problrem given a value for the bargaining power.

The use uf this library allows us to easily change the parametrization of the problem and rely on a robust solver to find solutions in multi-criteria non-linear restricted optimization problems. 

#### Static Model (JuMP)

#### Set up  (JuMP)

In [9]:
# download optimization libraries
#import Pkg; Pkg.add("JuMP")
#import Pkg; Pkg.add("Ipopt")
#import Pkg; Pkg.add("DataFrames")
#import Pkg; Pkg.add("ExcelFiles")

#### Libraries

In [10]:
using JuMP, Ipopt # Loading optimization libraries.
                  # Ipopt solve a nonlinear optimization problem.
using DataFrames  # Loading frame libraries.
using ExcelFiles  # Loading excel files libraries.

#### Utility Functions

In [11]:
#v(c,a,g=1,alpha=1)=-exp(g*(a-alpha*c))     # Declaration of v (agent utility function); 
                                            # g is the coefficient of risk aversion
                                            # alpha is a cost coefficient.

v(w, a, h=0.5) =  ((w)^(1-h))/(1-h) - a^2    # Declaration of v (agent utility function); 
                                                # h is a cost coefficient.

u(y, w) = y - w     # Declaration of u (Principal utility function).

u (generic function with 1 method)

#### Variables and lists  (JuMP)

In [21]:
A = [0.1 0.2]    # Actions set [al ah]
Y = [0.4 0.8]    # Outcomes set [yl yh]
f = [2/3 1/3; 1/3 2/3]    # Probability matrix  [yh|ah yl|ah; yh|al yl|al]
beta = 0.99   # Future discount factor
N = 100    # Number of intervals for the state variable
K= 1:100 
vl = v(0,A[2]) # State variable's lower limit 
vh = (1/3)*v(Y[1],A[1])+(2/3)*v(Y[2],A[1]) # State variable's upper limit
V = LinRange(vl, vh, N) # State variable's vector 
UF = [] # List of final values of the principal utility
AcOpt = [] # List of optimal actions for K
WHOpt = [] # List of hight optimal compensation for K
WLOpt = [] # List of low optimal compensation for K

Any[]

#### High Effort Model Specitication 

In [13]:
function Model_High_Effort(k, h)
    # A optimization model is created to incentive ah.
    model_high_effort = Model(
        with_optimizer(
            Ipopt.Optimizer, 
            tol = 1e-7, 
            max_iter = 1000, 
            print_level=1)) 
    
    # Declare the decision variables for the model: xl = wl, xh = wh.
    @variable(model_high_effort, xl >= 0.00001)  # xl = wl, low compensation.
    @variable(model_high_effort, xh >= 0.00001)  # xh = wh, hight compensation.
    
    # Declare the agent's utility function within the model.
    register(model_high_effort, :v, 2, v, autodiff=true)    # m, the model name
                                            # :v,  **********                                          
                                            # 2, number of variables
                                            # v, declare the agent's utility function 
                                            # autodiff,    **********
    register(model_high_effort, :u, 2, u, autodiff=true)  
    
    # Non-linear type expressions are declared, the expected utility of the agent, within the model. 
    EV_H = @NLexpression(model_high_effort, f[1,1]*v(xh,A[2])+f[1,2]*v(xl,A[2]))    # E(v|ah)
    EV_L = @NLexpression(model_high_effort, f[2,1]*v(xh,A[1])+f[2,2]*v(xl,A[1]))    # E(v|al)
    EU_H = @NLexpression(model_high_effort, f[1,1]*u(Y[2],xh) + f[1,2]*u(Y[1],xl))    # E(u|ah)
    
    # Objective function; expected utility of the principal given ah.
    @NLobjective(model_high_effort, Max,  EU_H) 
    
    @constraint(model_high_effort, xh<=Y[2])  # Financial capacity restriction for high compensation; wh<=yh.
    @constraint(model_high_effort, xl<=Y[1])  # Financial capacity restriction for low compensation; wl<=yl.
    
    @NLconstraint(model_high_effort, EV_H >= EV_L) # Incentive constraint to incentivize ah.
    @NLconstraint(model_high_effort, EV_H == V[k]) # Participation constraint to incentivize ah.
    
    # The problem posed for incentive ah is solved.
    JuMP.optimize!(model_high_effort) 
    
    # The optimal value of the function is saved given ah.
    uh = getobjectivevalue(model_high_effort)/(1-beta) 
    wh = value(xh)
    wl = value(xl)
    return uh, wh, wl, termination_status(model_high_effort)
end

Model_High_Effort (generic function with 1 method)

#### High Effort Model Specitication 

In [14]:
function Model_Low_Effort(k, h)
    # A optimization model is created to incentive al.
    model_low_effort = Model(
        with_optimizer(
            Ipopt.Optimizer, 
            tol = 1e-7, 
            max_iter = 1000,
            print_level=1))
    
    # Declare the decision variables for the model: xl = wl, xh = wh.
    @variable(model_low_effort, xl >= 0.00001)  # xl = wl, low compensation.
    @variable(model_low_effort, xh >= 0.00001)  # xh = wh, hight compensation.
    
    # Declare the agent's utility function within the model.
    register(model_low_effort, :v, 2, v, autodiff=true)    # model_low_effort, the model name
                                            # :v,  **********                                          
                                            # 2, number of variables
                                            # v, declare the agent's utility function 
                                            # autodiff,    **********
    register(model_low_effort, :u, 2, u, autodiff=true) # model_low_effort, the model name
                                            # :u,  **********                                          
                                            # 2, number of variables
                                            # u, declare the principal's utility function 
                                            # autodiff,    **********
    
    # Non-linear type expressions are declared, the expected utility of the agent, within the model. 
    EV_H = @NLexpression(model_low_effort, f[1,1]*v(xh,A[2]) + f[1,2]*v(xl,A[2])) #E(v|ah)
    EV_L = @NLexpression(model_low_effort, f[2,1]*v(xh,A[1]) + f[2,2]*v(xl,A[1])) #E(v|al)
    EU_L = @NLexpression(model_low_effort,f[2,1]*u(Y[2],xh) + f[2,2]*u(Y[1],xl)) # E(u|al))
    
    # Objective function; expected utility of the principal given al.
    @NLobjective(model_low_effort, Max, EU_L) 
    
    @constraint(model_low_effort, xh <= Y[2]) # Financial capacity restriction for high compensation; wh<=yh.
    @constraint(model_low_effort, xl <= Y[1]) # Financial capacity restriction for low compensation; wl<=yl.
    
    @NLconstraint(model_low_effort, EV_H <= EV_L)  # Incentive constraint to incentivize al.
    @NLconstraint(model_low_effort, EV_L == V[k])  # Participation constraint to incentivize al.
    
    # The problem posed for incentive al is solved.
    JuMP.optimize!(model_low_effort)
    
    # The optimal value of the function is saved given al.
    ul = getobjectivevalue(model_low_effort)/(1-beta) 
    wh = value(xh)
    wl = value(xl)
    return ul, wh, wl, termination_status(model_low_effort)
end

Model_Low_Effort (generic function with 1 method)

##### Routine (JuMP)

In [22]:
function run_static_model(V, K)
    
    UF = [] # List of final values of the principal utility
    AcOpt = [] # List of optimal actions for K
    WHOpt = [] # List of hight optimal compensation for K
    WLOpt = [] # List of low optimal compensation for K
    KFact = [] #list of feasible indexes 
    
    for k in K # For each k from 19 to 90 it is optimized.

        uval = 0 # Initial optimal utility of Principal for k.
        wl = 0 # Initial compensation if yl occurs.
        wh = 0 # Initial compensation if yh occurs.
        accopt = 0 # Initial optimal action.

        uh, xh, xl, stat_high = Model_High_Effort(k, h)

        # If the solution found is optimal and feasible, then:
        # The optimal value is saved; as well as the compensations scheme and the value ah for k.
        if (uh > uval) & (stat_high == MOI.LOCALLY_SOLVED) 
            uval = uh
            wh = xh
            wl = xl
            accopt = A[2]
        end

        ul, xh, xl, stat_low = Model_Low_Effort(k, h) 

        # If the solution found is optimal, for now u=uh, and feasible, then:
        # The optimal value is saved; as well as the compensations scheme and the value ah for k.
        if (ul > uval) & (stat_low == MOI.LOCALLY_SOLVED) 
            uval = ul
            wh = xh
            wl = xl
            accopt = A[1]
        end
        
        if uval != 0
            append!(KFact, k) # Save  k.
            append!(UF, uval) # Save the Value Function for k.
            append!(WLOpt, wl) # Save the optimal low compensation for k.
            append!(WHOpt, wh) # Save the optimal hight compensation for k.
            append!(AcOpt, accopt) # Save the optimal effort for k.
            
            # Results are printed for k
            println("K=",k,"   ",uval, "   ", wl ,"    ",wh,"    ", accopt)
        end

    end

    # The DataFrame is created
    DataEst = DataFrame(
        K = KFact, 
        Agent_Utility = V[KFact],
        Principal_Utility = UF,
        High_Compensation = WHOpt,
        Low_Compensation = WLOpt,
        Effort = AcOpt)
    return DataEst
end

K=5   66.51133446944618   1.0344616706686677e-5    0.0023248106499530827    0.2
K=6   66.44927251228852   0.00013271846585991422    0.003194553082741141    0.2
K=7   66.3734191698887   0.0003930086912497889    0.004202208106043682    0.2
K=8   66.2837742537073   0.0007912140026009709    0.005347779193089095    0.2
K=9   66.18033779157705   0.0013273343628742516    0.006631265944906199    0.2
K=10   66.0631098013092   0.0020013700002943177    0.008052667980214027    0.2
K=11   65.93209021039749   0.002813318368458477    0.009611987659807511    0.2
K=12   65.78727923546836   0.003763187140345972    0.01130921789780073    0.2
K=13   65.62867637657217   0.004850954979375004    0.013144376861728894    0.2
K=14   65.45628255666347   0.006076665407002572    0.015117428946545898    0.2
K=15   65.27009692482503   0.007440277464471182    0.01722840739538816    0.2
K=16   65.07011976470561   0.008941804804669298    0.01947730112708034    0.2
K=17   64.85635103910414   0.010581244875664942    0.02

#### Ploting 

#### Pareto Front Plot

In [ ]:
using DataFrames
using XLSX

SM = DataFrame(XLSX.readtable("Static_Model_JuMP.xlsx", "Sheet1")...)
UF = SM[:,"Principal_Utility"]

SM10 = DataFrame(XLSX.readtable("Static_Model_JuMP_h10.xlsx", "Sheet1")...)
UF10 = SM10[:,"Principal_Utility"]

SM25 = DataFrame(XLSX.readtable("Static_Model_JuMP_h25.xlsx", "Sheet1")...)
UF25 = SM25[:,"Principal_Utility"]

SM50 = DataFrame(XLSX.readtable("Static_Model_JuMP_h50.xlsx", "Sheet1")...)
UF50 = SM50[:,"Principal_Utility"]

SM75 = DataFrame(XLSX.readtable("Static_Model_JuMP_h75.xlsx", "Sheet1")...)
UF75 = SM75[:,"Principal_Utility"]

K = 19:90    # Feasible indices (In the article it is set V) for wang
K10 = 9:98   # Feasible indices (In the article it is set V) for h=10
K25 = 8:98   # Feasible indices (In the article it is set V) for h=25
K50 = 5:99   # Feasible indices (In the article it is set V) for h=50
K75 = 7:99   # Feasible indices (In the article it is set V) for h=75

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl1=plot(
    K,
    [UF],
    lw = 3,
    labels = "Wang",
    color = [:red ],
    title = "Pareto Frontier", 
    legend = true,
    xlabel = "Agent reservation utility, level k",
    ylabel =  "Principal's optimal expected utility") 
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    margin = 10mm) 
display(pl1)
savefig("Pareto_front_Static_Model_JuMP")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl1=plot(
    [K,K50],
    [UF, UF50],
    lw = 3,
    labels = ["Wang" "h=0.50"],
    color = [:red :green ],
    title = "Pareto Frontier", 
    legend = true,
    xlabel = "Agent reservation utility, level k",
    ylabel =  "Principal's optimal expected utility") 
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    margin = 10mm) 
display(pl1)
savefig("Pareto_front_Static_Model_JuMP_h50")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl1=plot(
    [K10, K25, K50, K75],
    [UF10, UF25, UF50, UF75],
    lw = 3,
    labels = ["h=0.10" "h=0.25" "h=0.50" "h=0.75"],
    color = [:blue :orange :green :purple],
    title = "Pareto Frontier", 
    legend = true,
    xlabel = "Agent reservation utility, level k",
    ylabel =  "Principal's optimal expected utility") 
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    margin = 10mm) 
display(pl1)
savefig("Pareto_front_Static_Model_JuMP_hs")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl1=plot(
    [K, K10, K25, K50, K75],
    [UF, UF10, UF25, UF50, UF75],
    lw = 3,
    labels = ["Wang" "h=0.10" "h=0.25" "h=0.50" "h=0.75"],
    color = [:red :blue :orange :green :purple],
    title = "Pareto Frontier", 
    legend = true,
    xlabel = "Agent reservation utility, level k",
    ylabel =  "Principal's optimal expected utility") 
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    margin = 10mm) 
display(pl1)
savefig("Pareto_front_Static_Model_JuMP_all")

#### Compensation Scheme Plot

In [ ]:
WLM =  SM[:,"Low_Compensation"]
WLM10 = SM10[:,"Low_Compensation"]
WLM25 = SM25[:,"Low_Compensation"]
WLM50 = SM50[:,"Low_Compensation"]
WLM75 = SM75[:,"Low_Compensation"]
WHM =  SM[:,"High_Compensation"]
WHM10 = SM10[:,"High_Compensation"]
WHM25 = SM25[:,"High_Compensation"]
WHM50 = SM50[:,"High_Compensation"]
WHM75 = SM75[:,"High_Compensation"]

K

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl2=scatter(
    [K],
    [WLM,WHM ], 
    markersize = 3,
    labels = ["wl Wang" "wh Wang"],
    #marker = [""]
    color = [:red :red ],
    shape = [:hline :+ :utri ],
    title = "Compensation Scheme",     
    legend = true,
    label = ["Wl" "Wh"],
    xlabel = "Agent reservation utility, level k",
    ylabel = "Compensation") #Se grafica K vs. Esquema de compensacion
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    legendfont = font(8), 
    titlefont = font(15),
    #legend=:bottomright,
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl2)
savefig("Compensation_Scheme_Static_Model_JuMP")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl2=scatter(
    [K, K, K50, K50],
    [WLM, WHM, WLM50, WHM50], 
    markersize = 4,
    labels = ["wl Wang" "wh Wang" "wl h=0.50" "wh h=0.50"],
    color = [:red :red :green :green],
    shape = [:hline :+  :hline :+ :utri ],
    title = "Compensation Scheme",     
    legend = true,
    label = ["Wl" "Wh"],
    xlabel = "Agent reservation utility, level k",
    ylabel = "Compensation") #Se grafica K vs. Esquema de compensacion
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    legendfont = font(8), 
    titlefont = font(15),
    #legend=:bottomright,
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl2)
savefig("Compensation_Scheme_Static_Model_JuMP_h50")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl2=scatter(
    [K10, K10, K25, K25, K50, K50, K75, K75], 
    [WLM10, WHM10, WLM25, WHM25, WLM50, WHM50, WLM75, WHM75], 
    markersize = 4,
    labels = ["wl h=0.10" "wh h=0.10" "wl h=0.25" "wh h=0.25" "wl h=0.50" "wh h=0.50" "wl h=0.75" "wh h=0.75"],
    color = [:blue :blue :orange :orange :green :green :purple :purple],
    shape = [:hline :+ :hline :+ :hline :+ :hline :+ :utri ],
    title = "Compensation Scheme",     
    legend = true,
    label = ["Wl" "Wh"],
    xlabel = "Agent reservation utility, level k",
    ylabel = "Compensation") #Se grafica K vs. Esquema de compensacion
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    legendfont = font(8), 
    titlefont = font(15),
    #legend=:bottomright,
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl2)
savefig("Compensation_Scheme_Static_Model_JuMP_hs")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

gra=cgrad([:orange, :blue], [0.1,0.3,0.8])

pl2=scatter(
    [K, K, K10, K10, K25, K25, K50, K50, K75, K75],
    [WLM,WHM, WLM10,WHM10, WLM25,WHM25, WLM50,WHM50, WLM75,WHM75], 
    markersize = 3,
    labels = ["wl Wang" "wh Wang" "wl h=0.10" "wh h=0.10" "wl h=0.25" "wh h=0.25" "wl h=0.50" "wh h=0.50" "wl h=0.75" "wh h=0.75"],
    color = [:red :red  :blue :blue :orange :orange :green :green :purple :purple],
    shape = [:hline :+ :hline :+ :hline :+ :hline :+  :hline :+  :utri ],
    title = "Compensation Scheme",     
    legend = true,
    label = ["Wl" "Wh"],
    xlabel = "Agent reservation utility, level k",
    ylabel = "Compensation",
    #palette =palette([:red , :green],10))
    #c=gra) #Se grafica K vs. Esquema de compensacion
    )
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    legendfont = font(8), 
    titlefont = font(15),
    #legend=:bottomright,
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl2)
savefig("Compensation_Scheme_Static_Model_JuMP_all")

#### Optimal Effort Plot

In [ ]:
OP = SM[:,"Effort"]
OP10 = SM10[:,"Effort"]
OP25 = SM25[:,"Effort"]
OP50 = SM50[:,"Effort"]
OP75 = SM75[:,"Effort"]

K

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl3=scatter(
    [K],
    [OP],
    markersize = 3,
    labels = ["Wang"],
    color = [:red :red ],
    title ="Optimal Effort", 
    label = false,
    xlabel = "Agent reservation utility, level k",
    ylabel = "Effort") #Se grafica K vs. accion sugerida
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl3)
savefig("Optimal_Effort_Static_Model_JuMP")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl3=scatter(
    [K, K50],
    [OP, OP50],
    markersize = 3,
    labels = ["Wang" "h=0.50"],
    color = [:red :green],
    title ="Optimal Effort", 
    label = false,
    xlabel = "Agent reservation utility, level k",
    ylabel = "Effort") #Se grafica K vs. accion sugerida
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl3)
savefig("Optimal_Effort_Static_Model_JuMP_h50")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl3=scatter(
    [K10, K25, K50, K75],
    [OP10, OP25, OP50, OP75],
    markersize = 3,
    labels = ["h=0.10" "h=0.25" "h=0.50" "h=0.75"],
    color = [:blue :orange :green  :purple],
    title ="Optimal Effort", 
    label = false,
    xlabel = "Agent reservation utility, level k",
    ylabel = "Effort") #Se grafica K vs. accion sugerida
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl3)
savefig("Optimal_Effort_Static_Model_JuMP_hs")

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
theme(:ggplot2)     # Graphics theme

pl3=scatter(
    [K, K10, K25, K50, K75],
    [OP, OP10, OP25, OP50, OP75],
    markersize = 3,
    labels = ["Wang" "h=0.10" "h=0.25" "h=0.50" "h=0.75"],
    color = [:red  :blue :orange :green :purple ],
    title ="Optimal Effort", 
    label = false,
    xlabel = "Agent reservation utility, level k",
    ylabel = "Effort") #Se grafica K vs. accion sugerida
plot!(size = (800,600),
    xtickfont = font(8), 
    ytickfont = font(8), 
    guidefont = font(10), 
    titlefont = font(15),
    legend = :topleft,
    margin = 10mm) #tamaño de letras
display(pl3)
savefig("Optimal_Effort_Static_Model_JuMP_all")